In [1]:
# uncomment below if you see a gribscan module not found error
#%pip install gribscan

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pylab as plt
import matplotlib.cm as cm
import cmocean.cm as cmo
from scipy.interpolate import CloughTocher2DInterpolator, LinearNDInterpolator, NearestNDInterpolator
import gribscan
import intake

## Split the large IFS-FESOM Cycle 3 dataset depending on the frequency of output (hourly and 6-hourly)

In [4]:
cat = intake.open_catalog("https://data.nextgems-h2020.eu/catalog.yaml")

In [5]:
cat.IFS

IFS:
  args:
    path: https://data.nextgems-h2020.eu/IFS/main.yaml
  description: IFS model output
  driver: intake.catalog.local.YAMLFileCatalog
  metadata:
    catalog_dir: https://data.nextgems-h2020.eu


In [6]:
# which IFS experiments are available? Which streams are in the IFS catalog for experiment 'IFS_4.4-FESOM_5-cycle3'?
print(list(cat.IFS))
print(list(cat.IFS['IFS_4.4-FESOM_5-cycle3']))

['IFS_9-FESOM_5-production', 'IFS_9-FESOM_5-production-hist', 'IFS_9-FESOM_5-production-old-gribscan', 'IFS_9-FESOM_5-production-hist-old-gribscan', 'IFS_9-FESOM_5-production-old-prepurge', 'IFS_4.4-FESOM_5-cycle3', 'IFS_9-FESOM_5-cycle3', 'IFS_9-NEMO_25-cycle3', 'IFS_28-NEMO_25-cycle3', 'IFS_28-FESOM_25-cycle3', 'IFS_4.4-FESOM_5-cycle3-nofastdata', 'IFS_4.4-FESOM_5-cycle3-fastdata', 'IFS_9-FESOM_5-production-hist-coupspinup', 'IFS_9-FESOM_5-production-hist-coupspinup-deepon', 'IFS_grids']
['2D_1h_native', '3D_1h6h_native', '2D_monthly_native', '3D_monthly_native', '2D_monthly_0.25deg', '3D_monthly_0.25deg', '2D_1h_0.25deg', '3D_1h6h_0.25deg', '2D_monthly_1deg', '3D_monthly_1deg', '2D_1h_1deg', '3D_1h6h_1deg', 'WAM_1h_native']


In [7]:
ds = cat.IFS['IFS_4.4-FESOM_5-cycle3']['3D_1h6h_native'].to_dask() # load 3D with mixed 1h and 6h data

/fastdata/bm1235/python_environments/hamburg_hackathon/lib/python3.10/site-packages/xarray/core/utils.py:494: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  warnings.warn(


In [40]:
# includes many NaNs
ds

<xarray.Dataset>
Dimensions:  (time: 43394, level: 12, value: 26306560)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 300 500 700 750 800 850 875 900 925 950 975 1000
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01
Dimensions without coordinates: value
Data variables:
    cc       (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    ciwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    clwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    crwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    cswc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    q        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    t        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    u        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    v        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    w        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    z        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    ~        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In [41]:
ds.t.time

<xarray.DataArray 'time' (time: 43394)>
array(['2020-01-20T00:00:00.000000000', '2020-01-20T01:00:00.000000000',
       '2020-01-20T02:00:00.000000000', ..., '2024-12-31T22:00:00.000000000',
       '2024-12-31T23:00:00.000000000', '2025-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01

In [43]:
# some variables are hourly on particular levels
ds.t.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T12:00:00.000000000'), level=700, value=10).values

array([246.23670959, 246.3993988 , 246.46313477, 246.68531799,
       247.09275818, 247.37767029, 247.42457581, 247.26951599,
       246.99485779, 246.59437561, 246.35830688, 246.23497009,
       246.14816284])

In [55]:
# while the others are 6-hourly (filling in the hourly steps with NaNs)
ds.z.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=700, value=10).values

array([           nan,            nan,            nan,            nan,
                  nan,            nan, 26449.48632812,            nan,
                  nan,            nan,            nan,            nan,
       26568.61132812])

## the 6-hourly dataset

In [51]:
# resample to take every 6th value, ignoring the first at time T00000 
ds_sampled6H = ds.isel(time=slice(None, None, 6)).isel(time=slice(1, None))

### test it (ds_sampled6H)

In [52]:
ds_sampled6H.z.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=700, value=10).values

array([26449.48632812, 26568.61132812])

In [53]:
ds_sampled6H.t.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=700, value=10).values

array([247.42457581, 246.14816284])

In [54]:
# this 6-hourly dataset works well and should not contain NaNs, please use this for the rechunking
ds_sampled6H

<xarray.Dataset>
Dimensions:  (time: 7232, level: 12, value: 26306560)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 300 500 700 750 800 850 875 900 925 950 975 1000
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20T06:00:00 ... 2024-12-31T23:00:00
Dimensions without coordinates: value
Data variables:
    cc       (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    ciwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    clwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    crwc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    cswc     (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    q        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    t        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    u        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    v        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    w        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    z        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    ~        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

## define the hourly datasets (4x, as the levels are different)

In [84]:
ds_z_1H = ds.z.sel(level=[300,500,850])
ds_z_1H

<xarray.DataArray 'z' (time: 43394, level: 3, value: 26306560)>
dask.array<getitem, shape=(43394, 3, 26306560), dtype=float64, chunksize=(1, 1, 26306560), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 300 500 850
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01
Dimensions without coordinates: value
Attributes: (12/16)
    paramId:                    129
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                isobaricInhPa
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfName:                     geopotential
    cfVarName:                  z
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [64]:
ds_t_1H = ds.t.sel(level=[700])
ds_t_1H

<xarray.DataArray 't' (time: 43394, level: 1, value: 26306560)>
dask.array<getitem, shape=(43394, 1, 26306560), dtype=float64, chunksize=(1, 1, 26306560), chunktype=numpy.ndarray>
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 700
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01
Dimensions without coordinates: value
Attributes: (12/16)
    paramId:                    130
    dataType:                   fc
    numberOfPoints:             26306560
    typeOfLevel:                isobaricInhPa
    stepUnits:                  1
    stepType:                   instant
    ...                         ...
    cfName:                     air_temperature
    cfVarName:                  t
    missingValue:               9999
    totalNumber:                0
    NV:                         0
    gridDefinitionDescription:  Gaussian Latitude/Longitude Grid

In [67]:
ds_uv_1H = ds.sel(level=[850])[['u', 'v']]
ds_uv_1H

<xarray.Dataset>
Dimensions:  (time: 43394, level: 1, value: 26306560)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 850
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01
Dimensions without coordinates: value
Data variables:
    u        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    v        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

In [71]:
ds = ds.rename({'~': 'var120'})
ds_w_1H = ds.sel(level=[500,850])[['var120', 'w']]
ds_w_1H

<xarray.Dataset>
Dimensions:  (time: 43394, level: 2, value: 26306560)
Coordinates:
    lat      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * level    (level) int64 500 850
    lon      (value) float64 dask.array<chunksize=(26306560,), meta=np.ndarray>
  * time     (time) datetime64[ns] 2020-01-20 2020-01-20T01:00:00 ... 2025-01-01
Dimensions without coordinates: value
Data variables:
    var120   (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
    w        (time, level, value) float64 dask.array<chunksize=(1, 1, 26306560), meta=np.ndarray>
Attributes:
    edition:            1
    centre:             ecmf
    centreDescription:  European Centre for Medium-Range Weather Forecasts
    subCentre:          0
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.IF...

### test them (ds_z_1H, ds_t_1H, ds_uv_1H, ds_w_1H)

In [74]:
ds_z_1H.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=300, value=10).values

array([81743.78125  , 81722.671875 , 81740.1953125, 81756.390625 ,
       81754.3203125, 81747.2578125, 81767.3671875, 81777.6171875,
       81790.3203125, 81819.9453125, 81841.34375  , 81841.65625  ,
       81860.015625 ])

In [75]:
ds_z_1H.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=500, value=10).values

array([49512.0703125 , 49506.3046875 , 49537.98046875, 49560.5703125 ,
       49564.11328125, 49559.27734375, 49573.01171875, 49585.94921875,
       49610.35546875, 49644.55859375, 49665.328125  , 49665.3515625 ,
       49675.44921875])

In [76]:
ds_z_1H.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=850, value=10).values

array([12549.85058594, 12530.01074219, 12543.23730469, 12551.99121094,
       12544.19140625, 12532.40917969, 12542.29394531, 12566.33984375,
       12593.2265625 , 12634.16894531, 12665.62792969, 12667.79980469,
       12674.22753906])

In [77]:
ds_t_1H.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=700, value=10).values

array([246.23670959, 246.3993988 , 246.46313477, 246.68531799,
       247.09275818, 247.37767029, 247.42457581, 247.26951599,
       246.99485779, 246.59437561, 246.35830688, 246.23497009,
       246.14816284])

In [78]:
ds_uv_1H.u.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=850, value=10).values

array([ 4.46942139,  5.06635284,  5.57159424,  6.47878265,  7.09181976,
        7.89427948,  8.93199921, 10.14372253, 10.18625641, 10.53778076,
        9.69914246,  9.04992676,  8.93826294])

In [79]:
ds_uv_1H.v.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=850, value=10).values

array([-0.5305481 ,  0.84033966,  1.47730255,  1.3139801 ,  0.6704483 ,
        0.40898895,  0.57275391,  0.06990814,  0.79915619,  1.63044739,
        2.1950531 ,  2.46411896,  2.68868256])

In [80]:
ds_w_1H.var120.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=500, value=10).values

array([-0.00742674, -0.01170921, -0.0054121 , -0.0096674 , -0.01831436,
       -0.01238728,  0.00085545,  0.00820351,  0.00858116,  0.01319695,
        0.00906372, -0.02273369,  0.00246334])

In [81]:
ds_w_1H.var120.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=850, value=10).values

array([-0.01475859, -0.00655079,  0.00085068, -0.00377178,  0.0012579 ,
       -0.00677299, -0.00304985, -0.00912094, -0.01225758, -0.00648594,
        0.00745583,  0.00578117,  0.00266933])

In [82]:
ds_w_1H.w.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=500, value=10).values

array([ 0.05466461,  0.07533264,  0.03135681,  0.05397034,  0.11917114,
        0.07841492, -0.01716614, -0.05604553, -0.074646  , -0.10438538,
       -0.06167603,  0.14788818, -0.0151825 ])

In [83]:
ds_w_1H.w.sel(time=slice('2020-01-20T00:00:00.000000000','2020-01-20T012:00:00.000000000'), level=850, value=10).values

array([ 0.1631279 ,  0.05368805, -0.03365326,  0.01210022, -0.04384613,
        0.05549622,  0.01173401,  0.10864258,  0.12382507,  0.05891418,
       -0.09339905, -0.0856781 , -0.032547  ])